In [2]:
import numpy as np
import torch
from typing import Union

In [3]:
a = np.array([[1,2,3],[8,5,2],[4,5,6]])
diff = a[:,None] - a[None,:]
np.linalg.norm(diff,axis=-1)

array([[0.        , 7.68114575, 5.19615242],
       [7.68114575, 0.        , 5.65685425],
       [5.19615242, 5.65685425, 0.        ]])

In [4]:
a = torch.tensor([[1,2,3],[8,5,2],[4,5,6]],dtype=torch.float32)
a.detach()
diff = a[:,None] - a[None,:]
torch.norm(diff,dim=-1)

tensor([[0.0000, 7.6811, 5.1962],
        [7.6811, 0.0000, 5.6569],
        [5.1962, 5.6569, 0.0000]])

In [18]:
def get_cosine_sim(vecs:Union[np.ndarray, torch.Tensor],rm_mean=False)->np.ndarray:
    '''
    vecs: is a N*D matrix contains N vectors of D dimension
    return a distance matrix sim, sim[i,j] is the cosine similarity between vector i and j
    '''
    if isinstance(vecs,torch.Tensor):
        vecs = vecs.detach()
        if rm_mean:
            vecs -= torch.mean(vecs,dim=0)
        sim = torch.nn.functional.cosine_similarity(vecs[:,None],vecs[None,:],dim=-1)
        return sim.detach().cpu().numpy()
    elif isinstance(vecs,np.ndarray):
        if rm_mean:
            vecs -= np.mean(vecs,dim=0)
        norm = np.linalg.norm(vecs,axis=-1)
        norm_prod = norm[None,:]*norm[:,None]
        dot = np.matmul(vecs,vecs.transpose())
        return dot/(norm_prod + 1e-6)
    else:
        raise TypeError("input must be an ndarray or tensor")

In [21]:
torch_vecs = torch.tensor([[1,2,3,4],[2,4,6,8],[-1,1,1,-1]],dtype=torch.float32)
np_vecs = np.array([[1,2,3,4],[2,4,6,8],[-1,1,1,-1]],dtype=np.float32)
get_cosine_sim(np_vecs,rm_mean=False)

array([[0.9999999 , 0.99999994, 0.        ],
       [0.99999994, 0.99999994, 0.        ],
       [0.        , 0.        , 0.99999976]], dtype=float32)

In [22]:
def get_space_align(covs:Union[np.ndarray, torch.Tensor])->np.ndarray:
    '''
    covs: is a N*D*D matrix contains N covariance matrices
    sim[i,j] = trace(covs[i]*covs[j])/(|covs[i]|*|covs[j]|)
    return a distance matrix sim, sim[i,j] is the cosine similarity between cov i and j
    '''
    if isinstance(covs,torch.Tensor):
        covs = covs.detach()
        mat_prod = torch.einsum("imn,jnm->ij",covs,covs)
        norm = torch.linalg.matrix_norm(covs)
        norm_prod = norm[None,:]*norm[:,None]
        sim = mat_prod/(norm_prod + 1e-6)
        return sim.detach().cpu().numpy()
    elif isinstance(covs,np.ndarray):
        mat_prod = np.einsum("imn,jnm->ij",covs,covs)
        # norm is a vecotor of shape (N,)
        norm = np.linalg.norm(covs,axis=(1,2))
        norm_prod = norm[None,:]*norm[:,None]
        sim = mat_prod/(norm_prod + 1e-6)
        return sim
    else:
        raise TypeError("input must be an ndarray or tensor")

In [24]:
cov0 = [[1,2],[2,1]]
cov1 = [[3,0],[0,3]]
cov2 = [[2,4],[4,2]]
torch_covs = torch.tensor([cov0,cov1,cov2],dtype=torch.float32)
get_space_align(torch_covs)

array([[0.9999999 , 0.44721356, 0.9999999 ],
       [0.44721356, 1.        , 0.44721356],
       [0.9999999 , 0.44721356, 1.        ]], dtype=float32)

In [25]:
np_covs = np.array([cov0,cov1,cov2],dtype=np.float32)
get_space_align(np_covs)

array([[0.9999999 , 0.44721356, 0.9999999 ],
       [0.44721356, 1.        , 0.44721356],
       [0.9999999 , 0.44721356, 1.        ]], dtype=float32)

In [26]:
def get_cov_traces(covs:Union[np.ndarray, torch.Tensor])->np.ndarray:
    '''
    covs: is a N*D*D matrix contains N covariance matrices
    sim[i,j] = trace(covs[i]*covs[j])/(|covs[i]|*|covs[j]|)
    return a distance matrix sim, sim[i,j] is the cosine similarity between cov i and j
    '''
    if isinstance(covs,torch.Tensor):
        covs = covs.detach()
        trace = torch.einsum("ijj->i",covs)
        return trace.detach().cpu().numpy()
    elif isinstance(covs,np.ndarray):
        trace = np.einsum("ijj->i",covs)
        return trace
    else:
        raise TypeError("input must be an ndarray or tensor")

In [27]:
get_cov_traces(torch_covs)

array([2., 6., 4.], dtype=float32)